In [ ]:
#from airflow import DAG
#from airflow.operators.python_operator import PythonOperator # type: ignore
from datetime import datetime,  date, timedelta
import pandas as pd # type: ignore
import random
import numpy as np
import os
#import json
import matplotlib.pyplot as plt

sales = [20621719, 39121200, 59650000,82700000, 109540000,145330000, 204800000, 296323000, 585650000, 978450000]
trend = [0.001, 0.002, 0.004, 0.005, 0.01, 0.02, 0.03, 0.05, 0.07, 0.08]
seasonal_component = [0.082, 0.103, 0.114, 0.149, 0.167, 0.220, 0.4, 0.45, 0.51, 0.6]
seasonal_peak_month = [1,2,6,7,9,12]

# Настройка генератора случайных чисел для воспроизводимости
np.random.seed(42)
random.seed(42)

""" создание списка локаций"""
location_list = []
for i in range(1,1501):
    location_list.append(f"LC{i:05d}")
#location_list

""" генерация продаж в указанных в спике sales пределах"""
sales_generated = []
i = 0
ii = 150
for n in range(0,10):
    for i in range(0,150):
        if n > 0:
          sales_n =  random.randint(sales[n-1], sales[n])
        else:
          sales_n =  random.randint(1, sales[n])
        sales_generated.append({
            "percentile": (n+1)/10,
            "index": i,
            "sales": sales_n
        })


# to check
# Filter for items where 'percentile' is 0.7
#percentile_7_items = [item for item in sales_generated if item['percentile'] == 0.7]
#print(percentile_7_items)

"""перемешиваем список локаций и содаем батчи по 150 локаций для того, чтобы присвоить им значения из списка trend """
""" создано с помощью Gemini """

# Ensure location_list is shuffled for random parts
random.shuffle(location_list)

# Calculate the size of each part
part_size = len(location_list) // 10

# Divide the shuffled list into 10 equal parts
divided_locations = [
    location_list[i * part_size : (i + 1) * part_size]
    for i in range(10)
]

# Print the first few elements of each part to demonstrate the division
#for i, part in enumerate(divided_locations):
#    print(f"Part {i+1} (first 5 elements): {part[:5]} ... (total {len(part)} elements)")

# You can access individual parts like: divided_locations[0], divided_locations[1], etc.
''' Gemini code end '''

""" присвоим значения trends """
trend_generated = []
for i, part in enumerate(divided_locations):
    for x in range(0,150):
        if i > 0:
          trend_n =  random.randint(int(trend[i-1]*10000), int(trend[i]*10000))
        else:
          trend_n =  random.randint(0, int(trend[i]*10000))
        trend_generated.append({
            "location": part[x],
            "trend": trend_n/10000
        })

#""" check"""
#trend_003 = [item for item in trend_generated if item['trend']==0.003]
#print(trend_003)

""" присвоим значения коэфф. сезонного прироста """
seas_component_generated = []
for i, part in enumerate(divided_locations):
    for x in range(0,150):
        if i > 0:
          seasonal_component_n =  random.randint(int(seasonal_component[i-1]*10000), int(seasonal_component[i]*10000))
        else:
          seasonal_component_n  =  random.randint(0, int(seasonal_component[i]*10000))
        seas_component_generated.append({
            "location": part[x],
            "seasonal_component": seasonal_component_n/10000,
            "seasonal_peak_month": random.choice(seasonal_peak_month)
        })

""" объединяем входные данные по локациям для генерации данных timeserires """
locations_df = pd.DataFrame(location_list, columns = ['location'])
sales_df = pd.DataFrame(sales_generated)
trend_df = pd.DataFrame(trend_generated)
seasonal_comp_df = pd.DataFrame(seas_component_generated)
joint_data = pd.merge(locations_df, sales_df[['sales']], right_index=True, left_index=True, how = "left")
joint_data = pd.merge(joint_data, trend_df, on = "location", how = "left")
joint_data = pd.merge(joint_data, seasonal_comp_df, on = "location", how = "left")
#joint_data.head()

""" генерируем данные """
"""код сгененрирован с помощью Gemini: """

def generate_sales_data(num_months, total_sales_target, trend_growth_percentage, seasonal_amplitude_percentage, seasonal_period_months, seasonal_peak_month):
    """
    Generates time series data for sales with a linear trend, seasonal component, and random noise,
    scaled to a specific total sum.

    Args:
        num_months (int): Number of months to generate data for.
        total_sales_target (float): The desired total sum of sales over all months.
        trend_growth_percentage (float): Total linear growth percentage over the period (e.g., 0.11 for 11%).
        seasonal_amplitude_percentage (float): The amplitude of the seasonal component as a percentage of the trend.
        seasonal_period_months (int): The period of the seasonal cycle in months (e.g., 12 for annual).
        seasonal_peak_month (int): The month (1-indexed) where the seasonal component peaks (e.g., 7 for July).

    Returns:
        tuple: A tuple containing:
            - time_series (numpy.ndarray): Array of month indices.
            - sales_data (numpy.ndarray): Array of generated sales data.
    """
    time_series = np.arange(num_months)

    # Calculate base sales for trend (starting point for linear growth)
    # Assuming the average sales would be total_sales_target / num_months
    # and the trend grows from (1 - trend_growth_percentage/2) to (1 + trend_growth_percentage/2)
    # For 11% total growth, starting point should be lower than average.
    # Let's simplify: base sales can be adjusted after scaling.

    # Linear trend component: starts at 1 and grows by trend_growth_percentage over num_months
    linear_trend_component = 1 + (time_series / (num_months - 1)) * trend_growth_percentage

    # Seasonal component: sine wave
    # Phase shift to make the peak occur in seasonal_peak_month
    # A sine wave peaks at pi/2. We want peak at `seasonal_peak_month - 1` (0-indexed).
    # (time * 2 * pi / period + phase_shift) -> phase_shift = pi/2 - (seasonal_peak_month - 1) * 2 * pi / period
    phase_shift = (np.pi / 2) - (((seasonal_peak_month - 1) % seasonal_period_months) * (2 * np.pi / seasonal_period_months))
    seasonal_component = seasonal_amplitude_percentage * np.sin(time_series * (2 * np.pi / seasonal_period_months) + phase_shift)

    # Random noise component
    noise = np.random.normal(loc=0, scale=0.05, size=num_months) # Small random noise

    # Combine components to get raw sales
    # Initial raw sales, before scaling to target total
    raw_sales = linear_trend_component + seasonal_component + noise
    raw_sales = np.maximum(0.1, raw_sales) # Ensure no negative sales, small positive base

    # Scale the raw sales to meet the total_sales_target
    current_total_sales = np.sum(raw_sales)
    scaling_factor = total_sales_target / current_total_sales
    sales_data = raw_sales * scaling_factor

    return time_series, sales_data

# Set random seed for reproducibility
np.random.seed(42)

num_months = 44
seasonal_period_months = 12
final_data = []
for iii in range(len(joint_data)):
    time_series_indices, sales_data = generate_sales_data(
        num_months,
        joint_data['sales'][iii],
        joint_data['trend'][iii],
        joint_data['seasonal_component'][iii],
        seasonal_period_months,
        joint_data['seasonal_peak_month'][iii]
)
    # Generate actual dates starting from January 1, 2022
    start_date = date(2022, 1, 1)
    #dates = [start_date + timedelta(days=30.5 * i) for i in time_series_indices]
    dates = pd.date_range(start_date, periods = num_months,  freq ='ME').to_period('M').astype('str')+"-01"
    final_data.append({
        "location":joint_data['location'][iii],
        "date": dates,
        "sales_gen": sales_data
    })

final_data_df = pd.DataFrame(final_data)

final_data_2 = []
for y in range(len(final_data_df)):
    for i in range(num_months):
        final_data_2.append({
           "location": final_data_df['location'][y],
           "date": final_data_df['date'][y][i],
           "sales": final_data_df['sales_gen'][y][i]
        })

final_data_df_2 = pd.DataFrame(final_data_2)